<main style="font-family: TeX Gyre Termes; font-size: 1.2rem">

### MEX #7 - Geyzson Kristoffer
SN:2023-21036

https://uvle.upd.edu.ph/mod/assign/view.php?id=547271

<hr>

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tqdm as notebook_tqdm

In [10]:
diabetes = pd.read_csv('diabetes_data_upload.csv')
diabetes.head()

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,40,Male,No,Yes,No,Yes,No,No,No,Yes,No,Yes,No,Yes,Yes,Yes,Positive
1,58,Male,No,No,No,Yes,No,No,Yes,No,No,No,Yes,No,Yes,No,Positive
2,41,Male,Yes,No,No,Yes,Yes,No,No,Yes,No,Yes,No,Yes,Yes,No,Positive
3,45,Male,No,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes,No,No,No,No,Positive
4,60,Male,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Positive


In [11]:
diabetes.shape

(520, 17)

In [12]:
diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Age                 520 non-null    int64 
 1   Gender              520 non-null    object
 2   Polyuria            520 non-null    object
 3   Polydipsia          520 non-null    object
 4   sudden weight loss  520 non-null    object
 5   weakness            520 non-null    object
 6   Polyphagia          520 non-null    object
 7   Genital thrush      520 non-null    object
 8   visual blurring     520 non-null    object
 9   Itching             520 non-null    object
 10  Irritability        520 non-null    object
 11  delayed healing     520 non-null    object
 12  partial paresis     520 non-null    object
 13  muscle stiffness    520 non-null    object
 14  Alopecia            520 non-null    object
 15  Obesity             520 non-null    object
 16  class               520 no

In [13]:
diabetes.describe()  

,Age
count,520.000000
mean,48.028846
std,12.151466
min,16.000000
25%,39.000000
50%,47.500000
75%,57.000000
max,90.000000


In [14]:
# change the categorical data to numerical data
diabetes

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,40,Male,No,Yes,No,Yes,No,No,No,Yes,No,Yes,No,Yes,Yes,Yes,Positive
1,58,Male,No,No,No,Yes,No,No,Yes,No,No,No,Yes,No,Yes,No,Positive
2,41,Male,Yes,No,No,Yes,Yes,No,No,Yes,No,Yes,No,Yes,Yes,No,Positive
3,45,Male,No,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes,No,No,No,No,Positive
4,60,Male,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,39,Female,Yes,Yes,Yes,No,Yes,No,No,Yes,No,Yes,Yes,No,No,No,Positive
516,48,Female,Yes,Yes,Yes,Yes,Yes,No,No,Yes,Yes,Yes,Yes,No,No,No,Positive
517,58,Female,Yes,Yes,Yes,Yes,Yes,No,Yes,No,No,No,Yes,Yes,No,Yes,Positive
518,32,Female,No,No,No,Yes,No,No,Yes,Yes,No,Yes,No,No,Yes,No,Negative


In [16]:
import optuna
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import make_scorer, f1_score, precision_score, recall_score
from ast import literal_eval

# Load your data
df = diabetes.copy()

# Assuming 'class' is the target variable
df['class'] = df['class'].map({'Positive': 1, 'Negative': 0})
X = df.drop('class', axis=1)
y = df['class']

# Encode categorical variables
le = LabelEncoder()
X = X.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')

# Split your data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features (important for models like SVM, kNN, and MLP)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the objective function
def objective(trial):
    classifier_name = trial.suggest_categorical('classifier', ['MLPClassifier', 'RandomForest', 'XGBoost', 'LogisticRegression', 'NaiveBayes', 'SVM', 'kNN'])
    
    # Hyperparameters spaces for each model
    if classifier_name == 'MLPClassifier':
        hidden_layer_sizes_str = trial.suggest_categorical('hidden_layer_sizes', ['(50, 50, 50)', '(50, 100, 50)', '(100,)'])
        hidden_layer_sizes = eval(hidden_layer_sizes_str)
        params = {
            'hidden_layer_sizes': hidden_layer_sizes,
            'activation': trial.suggest_categorical('activation', ['tanh', 'relu']),
            'solver': trial.suggest_categorical('solver', ['sgd', 'adam']),
            'max_iter': trial.suggest_int('max_iter', 1000, 2000), 
            'batch_size': trial.suggest_int('batch_size', 1, 100),
            'learning_rate_init': trial.suggest_float('learning_rate_init', 1e-5, 1e-2),
            'alpha': trial.suggest_float('alpha', 1e-5, 1e-2),
            'shuffle': trial.suggest_categorical('shuffle', [True, False]),
            'tol': trial.suggest_float('tol', 1e-5, 1e-2),
            'momentum': trial.suggest_float('momentum', 1e-5, 1e-2),
            'early_stopping': trial.suggest_categorical('early_stopping', [True, False]),
        }
        model = MLPClassifier(**params)
        
    elif classifier_name == 'RandomForest':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
            'max_depth': trial.suggest_int('max_depth', 1, 50),
            'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 14),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 14),
            'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),

        }
        model = RandomForestClassifier(**params)
        
    elif classifier_name == 'XGBoost':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
            'max_depth': trial.suggest_int('max_depth', 1, 50),
            'learning_rate': trial.suggest_float('learning_rate', 1e-8, 1.0),
            'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0),
            'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1e-6, 1e6),

        }
        model = xgb.XGBClassifier(**params)
        
    elif classifier_name == 'LogisticRegression':
        params = {
            'C': trial.suggest_float('C', 1e-8, 1e2),
            'max_iter': trial.suggest_int('max_iter', 1000, 2000),
        }
        model = LogisticRegression(**params)
        
    elif classifier_name == 'NaiveBayes':
        params = {
            'var_smoothing': trial.suggest_float('var_smoothing', 1e-8, 1e-2),
        }
        model = GaussianNB(**params)
        
    elif classifier_name == 'SVM':
        params = {
            'C': trial.suggest_float('C', 1e-8, 1e2),
            'kernel': trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid']),
            'degree': trial.suggest_int('degree', 1, 10),
            'gamma': trial.suggest_categorical('gamma', ['scale', 'auto']),
        }
        model = SVC(**params)
        
    elif classifier_name == 'kNN':
        params = {
            'n_neighbors': trial.suggest_int('n_neighbors', 1, 50),
            'weights': trial.suggest_categorical('weights', ['uniform', 'distance']),
            'algorithm': trial.suggest_categorical('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute']),
            'leaf_size': trial.suggest_int('leaf_size', 1, 50),
        }
        model = KNeighborsClassifier(**params)

    f1_scorer = make_scorer(f1_score, average='weighted')
    precision_scorer = make_scorer(precision_score)
    recall_scorer = make_scorer(recall_score)

    # 10-fold cross-validation
    # score = cross_val_score(model, X_train, y_train, n_jobs=-1, cv=10, scoring=f1_scorer).mean()
    score = cross_val_score(model, X_train, y_train, n_jobs=-1, cv=10).mean()
    return score

# Create a study object and specify the optimization direction
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Retrieve the best model
best_trial = study.best_trial
print(f"Best trial final score: {best_trial.value}")
for key, value in best_trial.params.items():
    print(f"{key}: {value}")

# Train the best model on the full training data and evaluate it on the test data
best_classifier = best_trial.params['classifier']


[I 2024-01-23 01:09:09,359] A new study created in memory with name: no-name-4b088767-43e2-47d9-9ad2-95c4857ccb23


[I 2024-01-23 01:09:09,673] Trial 0 finished with value: 0.9301974448315912 and parameters: {'classifier': 'MLPClassifier', 'hidden_layer_sizes': '(50, 100, 50)', 'activation': 'tanh', 'solver': 'adam', 'max_iter': 1270, 'batch_size': 76, 'learning_rate_init': 0.0008420695138320996, 'alpha': 0.007457946090759756, 'shuffle': False, 'tol': 0.00892647370554132, 'momentum': 0.0019367939573782054, 'early_stopping': False}. Best is trial 0 with value: 0.9301974448315912.
[I 2024-01-23 01:09:09,730] Trial 1 finished with value: 0.8558652729384436 and parameters: {'classifier': 'SVM', 'C': 32.76396330951154, 'kernel': 'sigmoid', 'degree': 10, 'gamma': 'auto'}. Best is trial 0 with value: 0.9301974448315912.
[I 2024-01-23 01:09:09,874] Trial 2 finished with value: 0.9639372822299652 and parameters: {'classifier': 'MLPClassifier', 'hidden_layer_sizes': '(100,)', 'activation': 'relu', 'solver': 'adam', 'max_iter': 1804, 'batch_size': 35, 'learning_rate_init': 0.005921063087388748, 'alpha': 0.0059

Best trial final score: 0.9759581881533101
classifier: XGBoost
n_estimators: 905
max_depth: 18
learning_rate: 0.562605408864118
booster: dart
reg_alpha: 0.11584891505972716
reg_lambda: 0.9212566499674906
scale_pos_weight: 959441.679020013


In [23]:
# Retrieve the best model hyperparameters
best_params = study.best_trial.params
best_params

{'classifier': 'XGBoost',
 'n_estimators': 905,
 'max_depth': 18,
 'learning_rate': 0.562605408864118,
 'booster': 'dart',
 'reg_alpha': 0.11584891505972716,
 'reg_lambda': 0.9212566499674906,
 'scale_pos_weight': 959441.679020013}

In [26]:
best_trial = study.best_trial
best_classifier = best_trial.params['classifier']
best_classifier

'XGBoost'

In [19]:
# Create the best model
if best_params['classifier'] == 'MLPClassifier':
    model = MLPClassifier(**{k: v for k, v in best_params.items() if k != 'classifier'})
elif best_params['classifier'] == 'RandomForest':
    model = RandomForestClassifier(**{k: v for k, v in best_params.items() if k != 'classifier'})
elif best_params['classifier'] == 'XGBoost':
    model = xgb.XGBClassifier(**{k: v for k, v in best_params.items() if k != 'classifier'})
elif best_params['classifier'] == 'LogisticRegression':
    model = LogisticRegression(**{k: v for k, v in best_params.items() if k != 'classifier'})
elif best_params['classifier'] == 'NaiveBayes':
    model = GaussianNB()
elif best_params['classifier'] == 'SVM':
    model = SVC(**{k: v for k, v in best_params.items() if k != 'classifier'})
elif best_params['classifier'] == 'kNN':
    model = KNeighborsClassifier(**{k: v for k, v in best_params.items() if k != 'classifier'})

print('fitting model...')
# Train the model on the full training data
model.fit(X_train, y_train, verbose=3)

print('evaluating model...')
# Evaluate the model on the test data
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f"Test Set Evaluation Metrics:\nAccuracy: {accuracy}\nF1 Score: {f1}\nPrecision: {precision}\nRecall: {recall}")


fitting model...
evaluating model...
Test Set Evaluation Metrics:
Accuracy: 0.9903846153846154
F1 Score: 0.9903442711135019
Precision: 0.9905181623931624
Recall: 0.9903846153846154


In [34]:
def objective2(trial):

    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 50),
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 14),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 14),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),

    }
    model = RandomForestClassifier(**params)

    f1_scorer = make_scorer(f1_score, average='weighted')
    precision_scorer = make_scorer(precision_score)
    recall_scorer = make_scorer(recall_score)

    # 10-fold cross-validation
    score = cross_val_score(model, X_train, y_train, n_jobs=-1, cv=10, scoring=f1_scorer).mean()
    return score

# Create a study object and specify the optimization direction
study2 = optuna.create_study(direction='maximize')
study2.optimize(objective2, n_trials=100)

# Retrieve the best model
best_trial2 = study2.best_trial
print(f"Best trial final score: {best_trial2.value}")
for key, value in best_trial2.params.items():
    print(f"{key}: {value}")

# Train the best model on the full training data and evaluate it on the test data

[I 2024-01-23 01:47:54,962] A new study created in memory with name: no-name-965eeaff-8291-493c-890b-01f3d43afda9
[I 2024-01-23 01:47:56,817] Trial 0 finished with value: 0.9352344712573324 and parameters: {'n_estimators': 213, 'max_depth': 5, 'criterion': 'gini', 'min_samples_split': 10, 'min_samples_leaf': 3, 'bootstrap': True}. Best is trial 0 with value: 0.9352344712573324.
[I 2024-01-23 01:47:57,496] Trial 1 finished with value: 0.9078668861150068 and parameters: {'n_estimators': 413, 'max_depth': 35, 'criterion': 'gini', 'min_samples_split': 5, 'min_samples_leaf': 10, 'bootstrap': True}. Best is trial 0 with value: 0.9352344712573324.
[I 2024-01-23 01:47:57,687] Trial 2 finished with value: 0.9012327718838293 and parameters: {'n_estimators': 116, 'max_depth': 21, 'criterion': 'gini', 'min_samples_split': 5, 'min_samples_leaf': 14, 'bootstrap': True}. Best is trial 0 with value: 0.9352344712573324.
[I 2024-01-23 01:47:58,862] Trial 3 finished with value: 0.915144342299546 and para

Best trial final score: 0.9831326497607644
n_estimators: 634
max_depth: 42
criterion: entropy
min_samples_split: 6
min_samples_leaf: 1
bootstrap: False


In [35]:
best_classifier2 = best_trial2.params
best_classifier2

{'n_estimators': 634,
 'max_depth': 42,
 'criterion': 'entropy',
 'min_samples_split': 6,
 'min_samples_leaf': 1,
 'bootstrap': False}

In [36]:
model2 = RandomForestClassifier(**best_classifier2)
model2

RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=42,
                       min_samples_split=6, n_estimators=634)

In [37]:
model2.fit(X_train, y_train)

y_pred2 = model2.predict(X_test)
accuracy2 = accuracy_score(y_test, y_pred2)
f1_2 = f1_score(y_test, y_pred2, average='weighted')
precision2 = precision_score(y_test, y_pred2, average='weighted')
recall2 = recall_score(y_test, y_pred2, average='weighted')

print(f"Test Set Evaluation Metrics:\nAccuracy: {accuracy2}\nF1 Score: {f1_2}\nPrecision: {precision2}\nRecall: {recall2}")


Test Set Evaluation Metrics:
Accuracy: 0.9903846153846154
F1 Score: 0.9904222748776574
Precision: 0.9906674208144797
Recall: 0.9903846153846154


In [41]:
math_scores = pd.read_csv('student-mat.csv', delimiter=';')
math_scores

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,11,12,10


In [49]:
# Load your data
df = math_scores.copy()

y = df['G3']
X = df.copy().drop('G3', axis=1).copy()

# Encode categorical variables
le = LabelEncoder()
X = X.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')

# Split your data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features (important for models like SVM, kNN, and MLP)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [53]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
student_performance = fetch_ucirepo(id=320) 
  
# data (as pandas dataframes) 
X = student_performance.data.features 
y = student_performance.data.targets 
  
# metadata 
# print(student_performance.metadata) 
  
# variable information 
# print(student_performance.variables) 

X


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,yes,no,no,4,3,4,1,1,3,4
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,yes,no,5,3,3,1,1,3,2
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,yes,no,4,3,2,2,3,3,6
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,yes,3,2,2,1,1,5,0
4,GP,F,16,U,GT3,T,3,3,other,other,...,yes,no,no,4,3,2,1,2,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,MS,F,19,R,GT3,T,2,3,services,other,...,yes,yes,no,5,4,2,1,2,5,4
645,MS,F,18,U,LE3,T,3,1,teacher,services,...,yes,yes,no,4,3,4,1,1,1,4
646,MS,F,18,U,GT3,T,1,1,other,other,...,yes,no,no,1,1,1,1,1,5,6
647,MS,M,17,U,LE3,T,3,1,services,services,...,yes,yes,no,2,4,5,3,4,2,6


In [56]:
# import piplite
# await piplite.install('lazypredict')
# import lazypredict

# from lazypredict.Supervised import LazyClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

data = load_breast_cancer()
X = data.data
y= data.target

# X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)

# clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
# models,predictions = clf.fit(X_train, X_test, y_train, y_test)

# print(models)

In [57]:
X

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])